# ALFA-BD - Predição da Avaliação de Fluência em Leitura com SVM

## Leitura de texto dos Alunos do 2ª e 3º Anos do Fundamental

#### Autor: TS#03


### Algoritmo de SVM (Suport Vector Machine)

#### Bibliotecas

In [17]:
import glob
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_predict, train_test_split

%matplotlib inline

## Carregamento da Base de Dados

In [3]:
files = glob.glob('dataset/*.wav')
files

['dataset/9_3_publica_menina_regular.wav',
 'dataset/3_2_publica_menina_bom.wav',
 'dataset/11_2_publica_menino_regular.wav',
 'dataset/12_3_publica_menino_regular.wav',
 'dataset/6_2_publica_menino_ruim.wav',
 'dataset/1_2_particular_menina_regular.wav',
 'dataset/5_2_publica_menina_regular.wav',
 'dataset/7_3_particular_menina_bom.wav',
 'dataset/4_2_publica_menino_ruim.wav',
 'dataset/8_3_particular_menino_regular.wav',
 'dataset/2_2_particular_menino_regular.wav',
 'dataset/13_3_publica_menino_regular.wav',
 'dataset/10_3_publica_menina_regular.wav']

## Pré Processamento

Extração das features dos áudios utilizando MFCC. Mel-Frequency Cepstral Coefficients que analisa características espectrais de tempo curto, baseando-se no uso de uma escala de frequências que imita as características únicas do som que são perceptíveis pelo ouvido humano.

In [19]:
classes = {'bom': 0, 'regular': 1, 'ruim': 2}
n_mfcc = 12
X = []
y = []

for file in files:
    label = classes[file.split('_')[4].replace('.wav', '')]
    y.append(label)
    x, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(x, sr=sr, n_mfcc=n_mfcc).T
    X.append(mfcc)

In [20]:
X
#X = numpy.vstack(X)
#y = np.asarray(y)

[array([[-4.15572296e+02,  5.94668198e+01,  7.25192928e+00, ...,
          1.12206726e+01, -1.49723291e-01, -9.07895088e+00],
        [-4.18214539e+02,  5.74214783e+01,  7.85578918e+00, ...,
          5.40134907e+00, -2.18671370e+00, -6.62589073e+00],
        [-4.22665558e+02,  5.29848137e+01,  7.70487261e+00, ...,
          1.39866829e+00, -4.45052147e+00, -8.34281349e+00],
        ...,
        [-4.04519745e+02,  6.63586731e+01,  6.50827503e+00, ...,
          7.12554407e+00,  1.01433449e+01,  6.79685211e+00],
        [-3.96089081e+02,  7.24423981e+01,  5.10503197e+00, ...,
          1.27824335e+01,  1.31256809e+01,  3.58642173e+00],
        [-4.03354492e+02,  6.90361786e+01,  8.32829475e+00, ...,
          1.60754051e+01,  1.72080040e+01,  9.65972137e+00]], dtype=float32),
 array([[-4.3039944e+02,  2.9608818e+01,  5.9405375e+00, ...,
          7.5810432e+00,  6.6874056e+00,  5.8575816e+00],
        [-4.2801193e+02,  3.1999863e+01,  6.5276575e+00, ...,
          8.2327251e+00,  7.2329

## Treinamento e Teste

In [10]:
# Parametros para validação cruzada
C= [0.001, 0.01, 0.1, 1, 10, 100]
gamma = [0.001, 0.01, 0.1, 1, 10, 100]
params_grid = dict(clf__C=C, clf__gamma=gamma)

In [13]:
# Criação dos pipelines
pip_1 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', SVC())
])

pip_2 = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVC(kernel='poly'))
])

pip_3 = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVC(kernel='linear'))
])

SVM = [{'name': 'RBF(Standard)',        'pipeLine': pip_1},
       {'name': 'Polinomial(Standard)', 'pipeLine': pip_2},
       {'name': 'Linear(Standard)',     'pipeLine': pip_3},]

In [15]:
def crossValidade(clf, X, y):
    predictions = cross_val_predict(clf, X, y, cv=5)
    
    print (metrics.classification_report(y, predictions, [0, 1, 2]))
    print (f'Acurácia: {accuracy_score(predictions, y)}')
    
    cm = confusion_matrix(predictions, y)
    sns.heatmap(cm, annot=True)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

for pipe in SVM:
    crossValidade(pipe['pipeLine'], X, y)

    classifier = pipe['pipeLine'].fit(X_train, y_train)
       
    predictions = classifier.predict(X_test)
    
    print (metrics.classification_report(y_test, predictions, [0, 1, 2]))
    print (f'Acurácia: {accuracy_score(predictions, y_test)}')
    
    cm = confusion_matrix(predictions, y_test)
    sns.heatmap(cm, annot=True)

ValueError: Found input variables with inconsistent numbers of samples: [103390, 13]